In [4]:
'''
Scraping date from baseball-reference.com
This file is specifically for getting information that will be used to predict the probability of a batter hitting a 
single.
Notes:
    -Specific to the blue jays
    -Specific to the year
-Matthew Sims
'''

'\nScraping date from baseball-reference.com\nThis file is specifically for getting information that will be used to predict the probability of a batter hitting a \nsingle.\nNotes:\n    -Specific to the blue jays\n    -Specific to the year\n-Matthew Sims\n'

In [5]:
#list of libraries being imported
from bs4 import BeautifulSoup
import urllib
import collections
import re

In [36]:
#global variables
_home_team = []
_away_team = []
_date = []

In [7]:
#function used to convert the date from the form (year, month_name day_number) to (yyyymmdd)
def dateConversion(oldDate):
    newDate = ""
    month = ""
    day_num = ""
    dateSystemDict = {"Jan": "01",
                      "Feb": "02",
                      "Mar": "03",
                      "Apr": "04",
                      "May": "05",
                      "Jun": "06",
                      "Jul": "07",
                      "Aug": "08",
                      "Sep": "09",
                      "Oct": "10",
                      "Nov": "11",
                      "Dec": "12"}
    oldDate = oldDate.split(' ') #the default is space, but it is used here to be explicit
    splitSize = len(oldDate)
    month = dateSystemDict[oldDate[splitSize-2]]
    day_num = oldDate[splitSize-1]
    if (len(day_num) == 1):
        day_num = "0" + day_num
    elif (len(day_num) < 1 or len(day_num) > 2):
        print "error getting date from schedule table"
    newDate = year + month + day_num
    return newDate
        

In [8]:
#function to change the team abbreviation if needed
#the team abrs used in the table do not match up with the ones used in the url for boxscores
def teamAbrConversion(teamAbr):
    teamAbrDict = {"NYY":"NYA",
                   "TBR":"TBA",
                   "WSN":"WAS",
                   "NYM":"NYN",
                   "CHW":"CHA",
                   "KCR":"KCA",
                   "LAA":"ANA"}
    if (teamAbrDict.has_key(teamAbr)):
        return teamAbrDict[teamAbr]
    return teamAbr

In [37]:
#function to loop through a team's schedule a grab all the url needed to request boxscores
#hard-coded Toronto as the home team
def getListofBoxscoreURLs(schedule_table):
    listOfUrls = []
    #get a list of all the rows in the schedule table
    team_schedule_rows = schedule_table.find_all('tr')
    team_schedule_rows.pop(0)
    for a_row in team_schedule_rows:
        try:
            columns = a_row.find_all("td")
            base_url = "http://www.baseball-reference.com/boxes/"
            end_url = "0.shtml"
            oldTeamAbr = a_row.find(id="Click for standings on this date")
            global _date
            tempDate = dateConversion(a_row.find(title="Click for standings on this date").get_text())
            _date.append(tempDate)
            #find the home team by first looking in the row for an @ symbol
            TeamAbr = ""
            global _home_team
            global _away_team
            if ("@" in columns[5].get_text()):
                TeamAbr = teamAbrConversion(columns[6].get_text())
                _home_team.append(TeamAbr)
                _away_team.append(teamAbrConversion(abr1[team_index]))
            else:
                TeamAbr = teamAbrConversion(abr1[team_index])
                _away_team.append(TeamAbr)
                _home_team.append(teamAbrConversion(abr1[team_index]))
            url = base_url + TeamAbr + '/' + TeamAbr + tempDate + end_url 
            listOfUrls.append(url.encode('utf-8'))
        except:
            continue
    return listOfUrls

In [27]:
testURL = "http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml"
test_page = urllib.urlopen("http://www.baseball-reference.com/teams/TOR/2015-schedule-scores.shtml").read()
test_page = BeautifulSoup(test_page)

In [28]:
#function to check if event occurred
#fix later
def searchforEvent(count_label, event_list, columns):
    '''nameFound = False
    lastName = columns[7].get_text().split()
    lastName = lastName[len(lastName)-1]
    event_dict = {"lastName":lastName.encode('utf-8'), count_label: 1}
    for j in range(0, len(event_list)):
        if (event_list[j]["lastName"] == lastName):
            #print lastName
            nameFound = True
            event_list[j][count_label] += 1
    if (not nameFound):
        event_list.append(event_dict)
    return  event_list'''
    nameFound = False
    lastName = columns[7].get_text().split()
    lastName = lastName[len(lastName)-1]
    singleDict = {"lastName":lastName.encode('utf-8'), "singleCount": 1}
    for j in range(0, len(singlesList)):
        if (singlesList[j]["lastName"] == lastName):
            #print lastName
            nameFound = True
            singlesList[j]["singleCount"] += 1
    if (not nameFound):
        singlesList.append(singleDict)
    return singlesList

In [29]:
#function to scan through the play-by-play finding an event in the description to get the event stats for each plater
def getSingles(page):
    singlesList = []
    doublesList = []
    triplesList = []
    walksList = []
    home_runsList = []
    play_by_play_table = page.find(id="play_by_play")
    play_by_play_rows = play_by_play_table.find_all('tr')
    for i in range(0, len(play_by_play_rows)):
        try:
            columns = play_by_play_rows[i].find_all('td')
            event = columns[len(columns)-1].get_text()
            if ('Single' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                singleDict = {"lastName":lastName.encode('utf-8'), "singleCount": 1}
                for j in range(0, len(singlesList)):
                    if (singlesList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        singlesList[j]["singleCount"] += 1
                if (not nameFound):
                    singlesList.append(singleDict)
            elif ('Double' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                doubleDict = {"lastName":lastName.encode('utf-8'), "doubleCount": 1}
                for j in range(0, len(doublesList)):
                    if (doublesList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        doublesList[j]["doubleCount"] += 1
                if (not nameFound):
                    doublesList.append(doubleDict)
            elif ('Triple' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                tripleDict = {"lastName":lastName.encode('utf-8'), "tripleCount": 1}
                for j in range(0, len(triplesList)):
                    if (triplesList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        tripleList[j]["tripleCount"] += 1
                if (not nameFound):
                    triplesList.append(tripleDict)
            elif ('Walk' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                walkDict = {"lastName":lastName.encode('utf-8'), "walkCount": 1}
                for j in range(0, len(walksList)):
                    if (triplesList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        walkList[j]["walkCount"] += 1
                if (not nameFound):
                    walksList.append(walkDict)
            elif ('Home Run' in event):
                nameFound = False
                lastName = columns[7].get_text().split()
                lastName = lastName[len(lastName)-1]
                home_runDict = {"lastName":lastName.encode('utf-8'), "home_runCount": 1}
                for j in range(0, len(home_runsList)):
                    if (home_runsList[j]["lastName"] == lastName):
                        #print lastName
                        nameFound = True
                        home_runsList[j]["home_runCount"] += 1
                if (not nameFound):
                    home_runsList.append(home_runDict)
        except:
            continue
    return {"singles":singlesList, "doubles":doublesList, "triples":triplesList, "walks":walksList, "home_runs":
            home_runsList}
        

In [30]:
#function to return the opposing pitcher on the day
#getPitcher from the play by play table
def getPitcher(page, batter):
    play_by_play_table = page.find(id="play_by_play")
    play_by_play_rows = play_by_play_table.find_all('tr')
    for i in range(0, len(play_by_play_rows)):
        try:
            columns = play_by_play_rows[i].find_all('td')
            b_lastName = columns[7].get_text().split()
            b_lastName = b_lastName[len(b_lastName)-1]
            if (b_lastName in batter):
                p_lastName = columns[8].get_text().split()
                p_lastName = p_lastName[len(p_lastName)-1]
                return p_lastName.encode('utf-8')
        except:
            continue
    return "failed to find pitcher"
        

In [44]:
def getTeamPitchingStats(page, game_index):
    boxscore_batting_table = page.find(id=currentTeam[team_index] + "pitching") #testing
    pitcher_rows = boxscore_batting_table.find_all('tr')
    pitcher_rows.pop(0)
    gamePitchingStats = []
    for i in range (0,len(pitcher_rows)-1):
        record = {"name":"",
                  "team":"",
                  "date": "",
                  "IP": 0,
                  "H": 0,
                  "BF": 0,}
                    
        attributes = pitcher_rows[i].find_all('td')
        name = attributes[0].get_text().split(",")[0]

        record["name"] = name.strip().encode('utf-8')
        record["team"] =currentTeam[team_index]
        record["date"] = _date[game_index]        
        record["IP"] = (attributes[1].get_text()).encode('utf-8')
        record["H"] = (attributes[2].get_text()).encode('utf-8')
        record["BF"] = (attributes[9].get_text()).encode('utf-8')
        gamePitchingStats.append(record)
    return gamePitchingStats

In [45]:
#function to get all urls from a team's schedule then get all the batter information
def getAllTeamPitchingStats(t_schedule):
    #variable to store all data records to be inputted into excel
    data = []
    #get all the urls
    urlList = getListofBoxscoreURLs(t_schedule)
    #open each boxscore page to grab info
    for i in range (0,len(urlList[0:2])):
        boxscore_page = urllib.urlopen(urlList[i]).read()
        boxscore_page = BeautifulSoup(boxscore_page)
        data = data + getPitchingStats(boxscore_page, i)
    return data

In [46]:
#hard-coded for the Toronto Blue Jays
#column accesses are hard_coded by index
#for boxscoreURL in urlList:
    #opening the boxscore page 
def getTeamBattingStats(page, listOfSingles, listOfDoubles, listOfTriples, listOfWalks, listOfHomeRuns, game_index):
    boxscore_batting_table = page.find(id=currentTeam[team_index] + "batting")
    batter_rows = boxscore_batting_table.find_all('tr')
    batter_rows.pop(0)
    gameBattingStats = []
    for batter_info in batter_rows:
        foundName = False
        record = {"name":"",
                  "PA": 0,
                  "OBP": 0,
                  "SLG": 0,
                  "singles": 0,
                  "doubles": 0,
                  "triples": 0,
                  "walks":0,
                  "home_runs":0,
                  "pitcher_name": "",
                  "pitcher_OBP": 0,
                  "defensive_efficiency": 0,
                  "batter_team": currentTeam[team_index],
                  "home_team": _home_team[game_index],
                  "away_team": _away_team[game_index],
                  "date": _date[game_index]}
        attributes = batter_info.find_all('td')
        name = attributes[0].get_text().strip().split(' ')
        name.pop(len(name)-1)
        record["name"] = ' '.join(name).strip().encode('utf-8')
        record["PA"] = (attributes[7].get_text()).encode('utf-8')
        record["OBP"] = (attributes[9].get_text()).encode('utf-8')
        record["SLG"] = (attributes[10].get_text()).encode('utf-8')
        record["pitcher_name"] = getPitcher(page, record["name"])
        if (record["pitcher_name"] != "failed to find pitcher" and record["PA"] != ""):
            for single in listOfSingles:
                if (single["lastName"] in record["name"]):
                    record["singles"] = single["singleCount"]
                    break
            for double in listOfDoubles:
                #print double
                if (double["lastName"] in record["name"]):
                    record["doubles"] = double["doubleCount"]
                    break
            for triple in listOfTriples:
                if (triple["lastName"] in record["name"]):
                    record["triples"] = triple["tripleCount"]
                    break
            for walk in listOfWalks:
                if (walk["lastName"] in record["name"]):
                    record["walks"] = walk["walkCount"]
                    break
            for home_run in listOfHomeRuns:
                if (home_run["lastName"] in record["name"]):
                    record["home_runs"] = home_run["home_runCount"]
                    break
            gameBattingStats.append(record)
    return gameBattingStats
    


In [52]:
#function to get all urls from a team's schedule then get all the batter information
def getAllTeamBattingStats(t_schedule):
    #variable to store all data records to be inputted into excel
    b_data = []
    p_data = []
    data = {"batter_data": b_data, "pitcher_data": p_data}
    #get all the urls
    urlList = getListofBoxscoreURLs(t_schedule)
    game_index = 0
    #open each boxscore page to grab info
    for url in urlList[0:6]:
        try:
            boxscore_page = urllib.urlopen(url).read()
            boxscore_page = BeautifulSoup(boxscore_page)
            events = getSingles(boxscore_page) 
            b_data = b_data + getTeamBattingStats(boxscore_page, events["singles"], events["doubles"], events["triples"],
                                                  events["walks"], events["home_runs"], game_index)
            p_data = p_data + getTeamPitchingStats(boxscore_page, game_index)
            print url
        except:
            print "fails:" + url
        game_index += 1
    data = {"batter_data": b_data, "pitcher_data": p_data}
    return data

In [53]:
#global variables
currentTeam = ["TorontoBlueJays", "NewYorkYankees", "BostonRedSox", "BaltimoreOrioles", "TampaBayRays", 
               "DetroitTigers", "NewYorkMets", "KansasCityRoyals", "WashingtonNationals", "TexasRangers",
               "OaklandAthletics", "MiamiMarlins", "PhiladelphiaPhillies", "HoustonAstros", "SeattleMariners",
               "MinnesotaTwins", "ChicagoWhiteSox", "ClevelandIndians", "AtlantaBraves", "AnaheimAngels"]
abr1 = ["TOR", "NYY", "BOS", "BAL", "TBR", "DET", "NYM", "KCR", "WSN", "TEX", "OAK", "MIA", "PHI", "HOU", "SEA",
        "MIN", "CHW", "CLE", "ATL", "LAA"]
year = "2015"
p_data = []
b_data = []
for team_index in range(0, len(currentTeam[0:1])):
    currentTeamURL = "http://www.baseball-reference.com/teams/" + abr1[team_index] + "/2015-schedule-scores.shtml"
    #print currentTeamURL
    #opening the team schedule page 
    #try:
    print currentTeam[team_index]
    team_schedule = urllib.urlopen(currentTeamURL).read()
    team_schedule = BeautifulSoup(team_schedule)
    team_schedule_table = team_schedule.find(id="team_schedule")
    data = (getAllTeamBattingStats(team_schedule_table))
    p_data = p_data + data["pitcher_data"]
    b_data = b_data + data["batter_data"]
    #except:
        #print "fail:" + currentTeamURL

TorontoBlueJays
http://www.baseball-reference.com/boxes/NYA/NYA201504060.shtml
http://www.baseball-reference.com/boxes/NYA/NYA201504080.shtml
http://www.baseball-reference.com/boxes/NYA/NYA201504090.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504100.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504110.shtml
http://www.baseball-reference.com/boxes/BAL/BAL201504120.shtml


In [54]:
import pandas as pd
import numpy as np

In [55]:
#print data
dfp = pd.DataFrame(p_data)
dfb = pd.DataFrame(b_data)
#dfp = pd.DataFrame(pdata)

In [64]:
dfp.to_excel('pitcher_data.xlsx', sheet_name='sheet1', index=False)
dfb.to_excel('batter_data.xlsx', sheet_name='sheet1', index=False)

In [56]:
print dfp


    BF  H    IP      date            name             team
0   23  3   6    20150406  Drew Hutchison  TorontoBlueJays
1    7  0   1.2  20150406      Aaron Loup  TorontoBlueJays
2    3  0   1.1  20150406   Miguel Castro  TorontoBlueJays
3   24  4   6.1  20150408     R.A. Dickey  TorontoBlueJays
4    2  0   0.2  20150408   Miguel Castro  TorontoBlueJays
5    3  2   0    20150408      Aaron Loup  TorontoBlueJays
6    4  1   0.1  20150408     Brett Cecil  TorontoBlueJays
7    2  0   0.2  20150408   Roberto Osuna  TorontoBlueJays
8   24  6   5.2  20150409   Daniel Norris  TorontoBlueJays
9    6  1   1.1  20150409   Roberto Osuna  TorontoBlueJays
10   4  0   1    20150409     Brett Cecil  TorontoBlueJays
11   3  0   1    20150409   Miguel Castro  TorontoBlueJays
12  26  8   6    20150410    Mark Buehrle  TorontoBlueJays
13   3  1   1    20150410   Liam Hendriks  TorontoBlueJays
14   5  3   1    20150410      Colt Hynes  TorontoBlueJays
15   4  1   1    20150410   Marco Estrada  TorontoBlueJa

In [155]:
print _home_team